In [8]:
import sys
sys.path.append("../data")
sys.path.append("../utils")
from spark_functions import create_SparkSession

In [9]:
spark = create_SparkSession()

In [6]:
os.listdir("../utils")

['.ipynb_checkpoints', 'spark_functions.py', 'string_functions.py']